In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aivsreal3/Test_Images/image_22.jpg
/kaggle/input/aivsreal3/Test_Images/image_53.jpg
/kaggle/input/aivsreal3/Test_Images/image_141.jpg
/kaggle/input/aivsreal3/Test_Images/image_69.jpg
/kaggle/input/aivsreal3/Test_Images/image_68.jpg
/kaggle/input/aivsreal3/Test_Images/image_175.jpg
/kaggle/input/aivsreal3/Test_Images/image_166.jpg
/kaggle/input/aivsreal3/Test_Images/image_153.jpg
/kaggle/input/aivsreal3/Test_Images/image_176.jpg
/kaggle/input/aivsreal3/Test_Images/image_51.jpg
/kaggle/input/aivsreal3/Test_Images/image_178.jpg
/kaggle/input/aivsreal3/Test_Images/image_88.jpg
/kaggle/input/aivsreal3/Test_Images/image_132.jpg
/kaggle/input/aivsreal3/Test_Images/image_4.jpg
/kaggle/input/aivsreal3/Test_Images/image_75.jpg
/kaggle/input/aivsreal3/Test_Images/image_124.jpg
/kaggle/input/aivsreal3/Test_Images/image_180.jpg
/kaggle/input/aivsreal3/Test_Images/image_179.jpg
/kaggle/input/aivsreal3/Test_Images/image_12.jpg
/kaggle/input/aivsreal3/Test_Images/image_16.jpg
/kaggle/inp

In [26]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import shutil
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image,UnidentifiedImageError
import matplotlib.pyplot as plt


#hp
test_size = 0.2
learning_rate=0.0001
num_epochs= 30
batch_size= 32
device=torch.device("cuda" if torch.cuda.is_available() else "cpu") 
data_dir = '/kaggle/input/mergeddata-aivsreal/merged_data'
real_images_path = os.path.join(data_dir, 'Real')
ai_images_path = os.path.join(data_dir, 'AI')

real_images = [os.path.join(real_images_path, f) for f in os.listdir(real_images_path) if f.endswith(('.jpg', '.png'))]
ai_images = [os.path.join(ai_images_path, f) for f in os.listdir(ai_images_path) if f.endswith(('.jpg', '.png'))]

image_paths = real_images + ai_images
labels = [1] * len(real_images) + [0] * len(ai_images) 



train_paths, valid_paths, train_labels, valid_labels = train_test_split(image_paths, labels, test_size=test_size, random_state=42)

test_images_path = '/kaggle/input/testimages-2/Test_Images'
test_images = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.endswith(('.jpg', '.png'))]




from torch.utils.data import Dataset
from PIL import Image, UnidentifiedImageError

class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        # Filter out invalid images during initialization
        valid_data = [
            (path, label) for path, label in zip(image_paths, labels)
            if self.is_valid_image(path)
        ]
        self.image_paths, self.labels = zip(*valid_data) if valid_data else ([], [])
        self.transform = transform

    def is_valid_image(self, img_path):
        try:
            Image.open(img_path).convert("RGB")
            return True
        except UnidentifiedImageError:
            print(f"Skipping invalid image: {img_path}")
            return False

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

class TestImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

       
        self.image_paths = [path for path in self.image_paths if self.is_valid_image(path)]

    def is_valid_image(self, img_path):
        try:
         
            Image.open(img_path).convert("RGB")
            return True
        except UnidentifiedImageError:
            print(f"Skipping invalid image: {img_path}")
            return False

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(img_path)




transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#dataset
train_dataset = CustomImageDataset(train_paths, train_labels, transform=transform)
valid_dataset = CustomImageDataset(valid_paths, valid_labels, transform=transform)
test_dataset = TestImageDataset(test_images, transform=transform)
#dataload
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Skipping invalid image: /kaggle/input/mergeddata-aivsreal/merged_data/AI/image_530.jpg
Skipping invalid image: /kaggle/input/mergeddata-aivsreal/merged_data/AI/image_326.jpg


In [27]:
classes = ('real','ai')
#main network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.drop = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,2)
       
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.drop(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [28]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)#learning rate optimization

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % 2000 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}')

    avg_train_loss = running_loss / n_total_steps
    print(f'Epoch [{epoch + 1}/{num_epochs}], Avg Training Loss: {avg_train_loss:.4f}')

    scheduler.step()
       
print('finished training')

     

Epoch [1/30], Avg Training Loss: 0.5996
Epoch [2/30], Avg Training Loss: 0.5549
Epoch [3/30], Avg Training Loss: 0.5477
Epoch [4/30], Avg Training Loss: 0.5515
Epoch [5/30], Avg Training Loss: 0.5187
Epoch [6/30], Avg Training Loss: 0.5096
Epoch [7/30], Avg Training Loss: 0.4936
Epoch [8/30], Avg Training Loss: 0.5274
Epoch [9/30], Avg Training Loss: 0.5076
Epoch [10/30], Avg Training Loss: 0.4946
Epoch [11/30], Avg Training Loss: 0.5069
Epoch [12/30], Avg Training Loss: 0.4987
Epoch [13/30], Avg Training Loss: 0.5110
Epoch [14/30], Avg Training Loss: 0.5003
Epoch [15/30], Avg Training Loss: 0.4866
Epoch [16/30], Avg Training Loss: 0.5067
Epoch [17/30], Avg Training Loss: 0.4681
Epoch [18/30], Avg Training Loss: 0.4881
Epoch [19/30], Avg Training Loss: 0.4845
Epoch [20/30], Avg Training Loss: 0.5013
Epoch [21/30], Avg Training Loss: 0.4868
Epoch [22/30], Avg Training Loss: 0.5083
Epoch [23/30], Avg Training Loss: 0.4693
Epoch [24/30], Avg Training Loss: 0.4922
Epoch [25/30], Avg Traini

In [29]:
with torch.no_grad():
    n_correct = 0
    n_sample = 0
    for images ,labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs,1)
        n_sample += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
acc  = 100.0 * n_correct/n_sample
print(acc)

77.68924302788845


In [30]:
predictions=[]
with torch.no_grad():
    
    for images, filenames in test_loader:
        images = images.to(device)  
        outputs = model(images)
        _,predicted_test = torch.max(outputs,1)
        for filename, label in zip(filenames, predicted_test):
            
            label_str = 'AI' if label.item() == 1 else 'Real'
            predictions.append((filename, label_str))

In [31]:
import pandas as pd

submission = pd.DataFrame(predictions, columns=['Id', 'Label'])



submission.to_csv('submission2.csv', index=False)

In [32]:
submission.to_csv('/kaggle/working/submission2.csv', index=False)